# Algebraic Simplification Using Rewrite.jl
## Harrison Grodin (CMU) and Yijia Chen (UCB)

https://github.com/HarrisonGrodin/Rewrite.jl

## Goal:
Create a term rewriting library capable of generating **canonical** rewrite systems and simplifying expressions using **custom identities**.

We're going to explore three sample domains:

1. Boolean Algebra      
2. Standard   Algebra    
3. Abstract Algebra

In [1]:
# add https://github.com/HarrisonGrodin/Rewrite.jl

using Rewrite

In [2]:
@term 2 + sin(x^3)

@term(2 + sin(x ^ 3))

In [3]:
collect(@term f(a, b, 3))

3-element Array{Any,1}:
 @term(a)
 @term(b)
 @term(3)

In [4]:
parse(@term f(x + inv(y), 1))

:(f(inv(y) + x, 1))

# Boolean Algebra

In [5]:
normalize(@term x | ((false | true) & x))

@term(x)

In [6]:
normalize(@term (!x | !x) & x)

@term(false)

In [7]:
normalize(@term (x | (x & y)) & (!y | false))

@term(!y & x)

# Standard Algebra

In [8]:
normalize(@term -1 * (x + 0))

@term(-x)

In [9]:
normalize(@term inv(inv(sin(x)^0 + 3))log(n))

@term(4 * log(n))

## Differentiation

Using imported rules from DiffRules.jl.

In [10]:
normalize(@term diff(x^2 + 5x, x))

@term(2 * one(x) * x + 5 * one(x) + log(x) * x ^ 2 * zero(x) + zero(x) * x)

In [11]:
using SpecialSets
x = Variable(:x, TypeSet(Number));

In [12]:
normalize(@term diff($x^2 + 5*$x, $x))

@term(2x + 5)

In [13]:
normalize(@term diff(f($x) + sin($x^2), $x))

@term(2 * cos(x ^ 2) * x + diff(f(x), x))

## Constraints

When rules only apply some of the time.

In [14]:
normalize(@term abs(x^2))

@term(x ^ 2)

In [15]:
normalize(@term abs(-(3^x)))

@term(3 ^ x)

In [16]:
normalize(@term abs(sin(x)))

@term(abs(sin(x)))

## Properties

Flat (associative) and orderless (commutative).

In [17]:
with_context(AlgebraContext(props=Dict(:f => [Flat]))) do
    @term f(a, f(f(b, c), d))
end

@term(f(a, b, c, d))

In [18]:
normalize(@term(g(x) + f(x)), @term RULES [
    f(x) + g(x) => h(x, x)
])

@term(h(x, x))

In [19]:
normalize(@term 2*(cos(x)*sin(x)))

@term(sin(2x))

# Abstract Algebra

## Group Theory

In [20]:
using Rewrite.Completion

>ᵣ = LPO((:i, 1), (:*, 2), (:e, 0))

set_context!(AlgebraContext());

In [21]:
axioms = @term AXIOMS [
    ((x * y) * z  , x * (y * z)  )
    (i(x) * x     , e()          )
    (e() * x      , x            )
];

![Critical Pair](./CriticalPair.png)

In [22]:
rules = complete(>ᵣ, axioms)

convert.(Pair, collect(rules))

10-element Array{Pair{Fn,B} where B,1}:
         @term(e() * x) => @term(x)            
        @term(i(x) * x) => @term(e())          
     @term((x * y) * z) => @term(x * (y * z))  
 @term(i(x) * (x * z₁)) => @term(z₁)           
         @term(x * e()) => @term(x)            
          @term(i(e())) => @term(e())          
         @term(i(i(x))) => @term(x)            
        @term(x * i(x)) => @term(e())          
 @term(x * (i(x) * z₃)) => @term(z₃)           
      @term(i(x₁ * y₁)) => @term(i(y₁) * i(x₁))

In [23]:
normalize(@term(y * ((x * (i(x) * z)) * e())), rules)

@term(y * z)

In [24]:
normalize(@term(((i(x) * y) * (i(y) * e())) * i(y)), rules)

@term(i(x) * i(y))

# Conclusion

Want more details? Come talk to us! And visit us at the...

## Poster Session II, on Friday!

Interested in learning more, using Rewrite, or contributing?

## https://github.com/HarrisonGrodin/Rewrite.jl

### References

 - “Term Rewriting and All That” (Baader and Nipkow)
 - “MatchPy” and related research (Krebber et al.)
 - “An Introduction to Knuth–Bendix Completion” (A. J. J. Dick)
 - “A complete proof of correctness of the Knuth-Bendix completion algorithm” (Gérard Huet)
 - `Combinatorics.jl` (JuliaMath organization)
 - `DiffRules.jl` (JuliaDiff organization)


# Questions?

# Bonus Content!

In [ ]:
# Boolean Rules

[
    @term RULES [
        x | false => x
        x & true  => x

        x & true  => true
        x & false => false

        x | x => x
        x & x => x

        x | (x & y) => x
        x & (x | y) => x

        x | !x => true
        x & !x => false

        !(!x) => x
    ];
    TRS(
        EvalRule(&),
        EvalRule(|),
        EvalRule(!),
    );
]

## Types

```julia
struct Variable{I<:AbstractSet} <: Term
    name::Symbol
    index::Int
    image::I
end
```

```julia
struct Constant{T} <: Term
    value::T
end
```

```julia
struct Fn <: Term
    name::Symbol
    args::Vector{Term}
end
```

### Future Improvements

(In addition to fixing bugs, improving documentation, and writing more descriptive tests...)

**Features:**
 - Derive term image sets based on wrapper functions
 - Convert Julia function calls on term objects to Fn objects
 - Named rules and automated theorem proving
 - Custom REPL mode
 - Color and LaTeX display modes
 - Support for infinite terms

**Improvement:**
 - Compatibility with existing libraries
 - Context construction and storage
 - Unions, set differences, and new sets in SpecialSets.jl
 - Efficiency of flat and orderless matching